In [ ]:
import concurrent.futures

# Define your graph of functions and dependencies

import time

def task(n):
    print(f"performing task {n}")
    if n == 2:
        time.sleep(2)
    if n == 2:
        raise ValueError("wtf")
    return n

graph = {
    'node1': {
        'dependencies': [], 
        'fn': task,
        'args': (1,)
    },
    'node2': {
        'dependencies': [], 
        'fn': task,
        'args': (2,)
    },
    'node3': {
        'dependencies': ['node2', 'node1'], 
        'fn': task,
        'args': (3,)
    },
    'node4': {
        'dependencies': ['node1'], 
        'fn': task,
        'args': (4,)
    }
}
import threading
lock = threading.Lock()

# Create a dictionary to store the results of each function
results = {}
errors = {}
def run_function(node):
    # Check if all dependencies have been completed
    # dependencies = graph[node]['dependencies']
    # for dependency in dependencies:
    #     if dependency not in results:
    #         raise ValueError(f"Dependency {dependency} of node {node} has not been completed")

    # Execute the function and store the result
    try:
        task_result = graph[node]['fn'](*graph[node]['args'])
        with lock:
            results[node] = task_result
            print(f"{node} complete; task_result: {task_result}")

    except Exception as e:
        task_result = e
        with lock:
            errors[node] = e


#     # Create a ThreadPoolExecutor with the maximum number of threads
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     # Create a dictionary to store the futures
#     futures = {}

#     # Submit the functions to the executor
#     for node in graph:
#         futures[node] = executor.submit(run_function, node)

#     # Wait for all futures to complete
#     concurrent.futures.wait(futures.values())

# # Access the results
# for node, future in futures.items():
#     result = future.result()
#     print(f"Result of {node}: {result}")

# Create a ThreadPoolExecutor with the maximum number of threads
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     # Create a dictionary to store the futures
#     futures = {}

#     # Create a dictionary to store the completed dependencies
#     completed_dependencies = {}

#     # Submit the functions to the executor
#     for node in graph:
#         dependencies = graph[node]['dependencies']
#         dependencies_completed = [completed_dependencies[dep] for dep in dependencies]
#         future = executor.submit(run_function, node)

#         # Wait for dependencies to complete before submitting the dependent task
#         if dependencies_completed:
#             print(f"dependency for {node} complete")
#             concurrent.futures.wait(dependencies_completed)
#         futures[node] = future

#         # Store the completed dependency future
#         completed_dependencies[node] = future

# # Access the results
# for node, future in futures.items():
#     result = future.result()
#     print(f"Result of {node}: {result}")

# Create a thread-safe queue to store the tasks
import queue
task_queue = queue.Queue()

# Add the initial tasks to the queue
for node in graph:
    task_queue.put(node)

# Set the maximum number of threads
max_threads = 5

# Create a ThreadPoolExecutor with the maximum number of threads
with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
    # Create a dictionary to store the futures
    futures = {}

    # Process the tasks from the queue
    while not task_queue.empty():
        node = task_queue.get()
        # print(f"evaluating node {node}")
        # Check if all dependencies have been completed
        dependencies = graph[node]['dependencies']
        dependencies_completed = all(dependency in results for dependency in dependencies)
        
        dependencies_errored = any(dependency in errors for dependency in dependencies)
        # print(f"dependencies complete: {dependencies_completed}")
        if dependencies_completed:
            print(f"dependencies complete for node {node}. Submitting.")
            # Submit the function to the executor
            future = executor.submit(run_function, node)
            futures[node] = future

            # # Add the dependent tasks to the queue
            # for dependent_node in graph:
            #     if node in graph[dependent_node]['dependencies']:
            #         task_queue.put(dependent_node)
        elif dependencies_errored:
            raise ValueError(f"dependencies for {node} errored")
        else:
            # print(f"dependencies not complete for node {node}. Submitting.")
            # Re-add the task to the queue for later processing
            task_queue.put(node)

print("waiting for completion...")
# Wait for all futures to complete
concurrent.futures.wait(futures.values())

# Access the results
for node, future in futures.items():
    result = future.result()
    print(f"Result of {node}: {result}")

In [ ]:
results

In [ ]:
import time
time.sleep(3)
print("done")

In [ ]:
arg = (1,)
task(*arg)